In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv("beyazperde_sinemalar_birleşim.csv")

In [3]:
data["oyuncu+yönetmen"] = data["oyuncular_y"] + ", " + data["yönetmen_y"]

In [4]:
data

,Unnamed: 0,başlık,orijinal_isim,özet_x,tür_x,vizyon_tarihi,oyuncular_x,yönetmen_x,resim_link,isim,tür_y,link,özet_y,yönetmen_y,oyuncular_y,birleşim,oyuncu+yönetmen
0,0,Grinin Elli Tonu,Fifty Shades of Grey,Bir edebiyat öğrencisi olan güzel Anastasia St...,"['Dram', 'Romantik', 'Gerilim']",13 Şubat 2015,"['Jamie Dornan', 'Dakota Johnson', 'Jennifer E...",['Sam Taylor-Johnson'],https://tr.web.img3.acsta.net/c_310_420/pictur...,Grinin Elli Tonu,"Dram,Romantik,Gerilim",https://www.sinemalar.com/film/207818/grinin-e...,"Edebiyat ögrencisi olan Anastasia Steele, genç...",Sam Taylor-Johnson,"Jamie Dornan,Dakota Johnson,Jennifer Ehle,Rita...",bir edebiyat öğrencisi olan güzel anastasia st...,"Jamie Dornan,Dakota Johnson,Jennifer Ehle,Rita..."
1,1,Örümcek-Adam Eve Dönüş Yok,Spider-Man: No Way Home,"Örümcek-Adam Eve Dönüş Yok, kimliği açığa Örüm...","['Aksiyon', 'Macera', 'Fantastik']",17 Aralık 2021,"['Tom Holland', ' Zendaya', 'Benedict Cumberba...",['Jon Watts'],https://tr.web.img2.acsta.net/c_310_420/pictur...,Örümcek-Adam: Eve Dönüş Yok,"Aksiyon,Macera,Bilim Kurgu",https://www.sinemalar.com/film/253225/spider-m...,Filmde Örümcek-Adam'ın gerçek kimliğinin Peter...,Jon Watts,"Tom Holland,Zendaya Coleman,Benedict Cumberbat...","örümcek-adam eve dönüş yok, kimliği açığa örüm...","Tom Holland,Zendaya Coleman,Benedict Cumberbat..."
2,2,Recep İvedik 6,Recep İvedik 6,"Recep İvedik 6, Konya'ya giderken kendisini Ke...",['Komedi'],9 Kasım 2019,"['Şahan Gökbakar', 'Nurullah Çelebi', 'Somer K...",['Togan Gökbakar'],https://tr.web.img4.acsta.net/c_310_420/pictur...,Recep İvedik 6,Komedi,https://www.sinemalar.com/film/253450/recep-iv...,"Recep İvedik serisinin 6. filminde, Konya'ya g...",Togan Gökbakar,Şahan Gökbakar,"recep i̇vedik 6, konya'ya giderken kendisini k...","Şahan Gökbakar, Togan Gökbakar"
3,3,Arabalar 2,Cars 2,Ünlü yarış arabası Şimşek McQueen ve eşsiz kur...,['Animasyon'],19 Ağustos 2011,"['Owen Wilson', ' Larry The Cable Guy', 'Micha...","['Brad Lewis', 'John Lasseter']",https://tr.web.img3.acsta.net/c_310_420/medias...,Arabalar 2,"Animasyon,Aile,Macera,Komedi",https://www.sinemalar.com/film/9354/arabalar-2,"Şimşek McQueen, yeni pit ekibi ve yeni ekip şe...",John Lasseter\n ...,"Yekta Kopan,Emily Mortimer,Sefa Zengin,Eddie I...",ünlü yarış arabası şimşek mcqueen ve eşsiz kur...,"Yekta Kopan,Emily Mortimer,Sefa Zengin,Eddie I..."
4,4,İhanet,Partir,"Suzanne (Kristin Scott Thomas), mesleğinde old...","['Dram', 'Romantik']",20 Mayıs 2011,"['Kristin Scott Thomas', 'Sergi López', 'Yvan ...",['Catherine Corsini'],https://tr.web.img4.acsta.net/c_310_420/pictur...,İhanet,"Dram,Romantik",https://www.sinemalar.com/film/43264/ihanet,"Suzanne kırklı yaşlarında, iki genç çocuk anne...",Catherine Corsini,"Kristin Scott Thomas,Sergi López,Philippe Laud...","suzanne (kristin scott thomas), mesleğinde old...","Kristin Scott Thomas,Sergi López,Philippe Laud..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5290,5290,Hababam Sınıfı Dokuz Doğuruyor,Hababam Sınıfı Dokuz Doğuruyor,Haylaz öğrencilerle dolu Hababam sınıfına doğu...,"['Dramatik komedi', 'Romantik']",None,"['Münir Özkul', 'Şener Şen', 'İlyas Salman']",['Kartal Tibet'],https://tr.web.img3.acsta.net/c_310_420/pictur...,Hababam Sınıfı Dokuz Doğuruyor,Komedi,https://www.sinemalar.com/film/817/hababam-sin...,"Hababam sınıfına, doğulu bir aşiret reisinin o...",Kartal Tibet,"Şener Şen,Münir Özkul,Adile Naşit,Muharrem Gür...",haylaz öğrencilerle dolu hababam sınıfına doğu...,"Şener Şen,Münir Özkul,Adile Naşit,Muharrem Gür..."
5291,5291,Büyük Oyun,Büyük Oyun,"Büyük Oyun, başına gelen talihsiz olaylar neti...",['Dram'],17 Eylül 2010,"['Haktan Pak', 'Suzan Genç', 'Serkan Genç']",['Atıl İnaç'],https://tr.web.img2.acsta.net/c_310_420/medias...,Büyük Oyun,"Dram,Aksiyon",https://www.sinemalar.com/film/27961/buyuk-oyun,ABD’nin Irak’ı işgâl sürecinde direnişçileri y...,Atıl İnaç,"Rana Cabbar,Serkan Genç,Serdal Genç,Nalan Kuru...","büyük oyun, başına gelen talihsiz olaylar neti...",

In [5]:
data.isnull().values.any()

True

In [6]:
data = data.fillna("")

In [7]:
model = AutoModelForTokenClassification.from_pretrained("savasy/bert-base-turkish-ner-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-ner-cased")
ner=pipeline('ner', model=model, tokenizer=tokenizer)

In [8]:
example = "hakkari'de uma thurman ve şener şen'in oynadığı film"

In [9]:
ner_results = ner(example)

In [10]:
array = []
start = 0
end = 0
kontrol = 0
for result in ner_results:
    if (result['entity'] == 'B-PER'):
        if kontrol == 0:
            kontrol +=1
            start = result['start']
            end = result['end']
        else:
            array.append(example[start:end])
            start = result['start']
            end = result['end']
            
    elif (result['entity'] == 'I-PER'):
        end = result['end']
array.append(example[start:end])

print(array)

['uma thurman', 'şener şen']


In [11]:
s = ''
for x in array:
    s += " " + x

s = s.strip()

In [12]:
s

'uma thurman şener şen'

In [13]:
def create_dataframe(matrix, tokens):
    doc_names = [f'doc_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

In [14]:
indexes = []
values = []
for x in range(len(data["oyuncu+yönetmen"])):
    a = [data["oyuncu+yönetmen"][x], s]
    count_vectorizer = CountVectorizer()
    vector_matrix = count_vectorizer.fit_transform(a)
    tokens = count_vectorizer.get_feature_names()
    vector_matrix.toarray()
    create_dataframe(vector_matrix.toarray(),tokens)
    cosine_similarity_matrix = cosine_similarity(vector_matrix)
    cosine = (create_dataframe(cosine_similarity_matrix,[str(x),'query']))
    if (cosine.iloc[0,1] > 0 ):
        indexes.append(x)
        values.append(cosine.iloc[0,1])

In [15]:
for i in range(len(indexes)):
    print(indexes[i], values[i])

103 0.12909944487358055
275 0.2886751345948129
296 0.2886751345948129
302 0.2886751345948129
568 0.14433756729740646
574 0.2886751345948129
610 0.25
666 0.14433756729740646
704 0.14433756729740646
1017 0.2886751345948129
1072 0.2886751345948129
1273 0.2886751345948129
1341 0.2886751345948129
1384 0.25
1387 0.2886751345948129
1388 0.2773500981126146
1408 0.1386750490563073
1509 0.14433756729740646
1569 0.14433756729740646
1608 0.2886751345948129
1611 0.2886751345948129
1714 0.2886751345948129
1731 0.2886751345948129
1736 0.2886751345948129
1745 0.14433756729740646
1924 0.20412414523193154
2013 0.14433756729740646
2042 0.2886751345948129
2089 0.2886751345948129
2094 0.14433756729740646
2131 0.2886751345948129
2156 0.1336306209562122
2196 0.14433756729740646
2218 0.2886751345948129
2240 0.2886751345948129
2259 0.14433756729740646
2273 0.14433756729740646
2275 0.2886751345948129
2419 0.2886751345948129
2513 0.2886751345948129
2523 0.2886751345948129
2579 0.2886751345948129
2600 0.144337567